In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


# Casting

**Learning Outcomes**

To understand:

+ Casting as a concept
+ Implicit casting
+ Using `$` to cast data to different datatypes
+ Casting textual data 

# Casting Methods

When working with data, it is often necessary to change the data from one type e.g. a time like `09:30:00` to another e.g. a datetime like `2020.05.19T09:30:00`. 

## Implicit casting

In many cases however we don't need to explicitly use the casting operator `$` and can instead just use simple operations to achieve the result we want.

In [3]:
show time:09:30:00
show type time
show dt:.z.d
show type dt

show timeStamp:time+dt
show type timeStamp

09:30:00
-18h
2025.02.17
-14h
2025.02.17D09:30:00.000000000
-12h


Other operators can also have these types of effects - consider for example what is returned when you divide two long values: 

In [4]:
4%2

2f


##### Exercise

What datatype do you expect from the output below? 

Datatype:Date

In [6]:
type 2020.09.01 + 2

-14h


Fill out below!

Datatype:

##### Exercise 

Repeat exercise for the expression below.


Datatype:Timespan

Here we have added a timespan to a time. The timespan is the more granular of the two types and this is returned.

In [9]:
09:00:00 + 2D00:00:00


2D09:00:00.000000000


Fill out below!

Datatype: 

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Generally in kdb+/q, when you add two types (if they're compatible) they will resolve to the datatype with the highest level of granularity possible to represent the two types. </i></p>

## Casting using `$`
Explicit casting in kdb+/q uses the [`$`](https://code.kx.com/q/ref/cast/) (aka [cast](https://code.kx.com/q/ref/cast/)) operator.

Assuming the data you wish to cast is not a textual datatype (like symbol or string), any of the following three approaches can be utilized by `$`to specify the desired type of the output:
1.	It's symbol name e.g `float
2.	It's character letter e.g. "f"
3.	It's short value (aka short name) e.g. 9h

### Numerical datatypes 
If we wanted to cast a list of longs to a float list, all three casting operations are equivalent:

In [10]:
x:10 20 30
`float$x               // 1 symbol name - functional form $[`float;x]
"f"$x                  // 2 character value - functional form $["f";x]
9h$x                   // 3 short value - functional form $[9h;x]  

10 20 30f
10 20 30f
10 20 30f


The output of all three operations can be checked for equivalency:

In [11]:
show symAndCharEquiv:$[`float;x]~$["f";x]  //you get the same results casting either way
show symAndShortEquiv:$[`float;x]~$[9h;x]

1b
1b


In the following example we see we can actually cast integers to a (date/time & bytes):

In [12]:
show x:0 4000 29999i
$[`date;x]       //dates start from 2001.01.01 
$[19h;x]         //time type numeric value - note this is a short numeric value
$["x";x]         //cast as a byte

2000.01.01 2010.12.14 2082.02.18
00:00:00.000 00:00:04.000 00:00:29.999
0x00a02f
0 4000 29999i


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:20px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Be careful going from more granular to less granular types e.g. from float 5.4 to a long - because the long type has a lower precision some data is lost in the casting, <code>`long$5.4</code> returns a value of <code>5</code>. A round-trip casting won't necessarily return the same value as started with <code>`float$`long$5.4</code> will return <code>5f</code>.</i></p>

##### Exercise
Cast the value 123.23 to the following types: 
+ Real
+ Boolean 

What do you think would happen if you cast the boolean value back to float? 

In [ ]:
"e"$123.23
"b"$123.23

In [ ]:
"f"$"b"$123.23  //notice casting to float is the last operation in the chain

In [14]:
// Write your code here
q:123.23
$[`real;q]
$[`boolean;q]

123.23e
1b


### Temporal datatypes  

We can operate in the same fashion as above on temporal datatypes.

In [18]:
d:.z.d
`long$d // number of dates from 1st Jan 2000
t:09:30:00.000
`long$t  //number of miliseconds from midnight
show ts:d+t
`long$ts //number of miliseconds from midnight 1st Jan 2020

9179
34200000
793099800000000000
2025.02.17D09:30:00.000000000


What do you expect as outputs from the below?

In [ ]:
`minute$17:55:01 //rounds down to the nearest minute
`uu$17:55:01 //can also use this method to cast to minute however it returns an integer
`minute$17:55:59   //note the second differences here- can see it clearer that it rounds down 
`uu$17:55:59 //can see that this method also rounds down
`hh$17:55:59 //there is no hour time type in kdb+, casts it to an int 

In [15]:
`minute$17:55:01
`minute$17:55:59   //note the second differences here
`hh$17:55:59

17:55
17:55
17i


##### Exercise 

Cast the time 16:30:00 to  each of the following types: 
+ Integer
+ Float
+ Minute
+ Second

In [ ]:
"i"$16:30:00 // int
"f"$16:30:00  // float

In [ ]:
//Minute options
`uu$16:30:00
`minute$16:30:00  

In [ ]:
//Second options:
`ss$16:30:00
`second$16:30:00  

In [19]:
// Write your code here
r:16:30:00
$[`int;r]
$[`float;r]
$[`minute;r]
$[`second;r]

59400i
59400f
16:30
16:30:00


# Casting Textual data

In kdb+/q there are two textual datatypes, Symbols and Strings (lists of characters). These need to be treated differently from other datatypes when casting either *to*, or *from* them. 

## Strings 
Strings in kdb+/q are really just lists of characters.

In [20]:
"s","e","e","?"

see?


### Casting *to* a String

Casting to a string in kdb+/q is actually quite easy as there is a keyword [`string`](https://code.kx.com/q/ref/string/) which will cast any other type into it's string representation: 

In [21]:
string .z.d
string 12.0
string 12.01     //note the difference from the one above!
string `totallyASymbol

2025.02.17
12
12.01
totallyASymbol


In [22]:
A:(1000;2000i;2020.01.01;.z.t);  /We can see A here is a general list, meaning it contains various datatypes
string A

"1000"
"2000"
"2020.01.01"
"06:13:01.261"


##### Exercise
Cast 1234 to a string

In [ ]:
string 1234

In [23]:
// Write your code here
string 1234

1234


### Casting *from* a String

As a starting point, lets begin by trying to cast a string in the same way that we cast our other datatypes: 

In [24]:
`float$"4222.001"    //hmmmmm

52 50 50 50 46 48 48 49f


Here kdb+/q has treated the string as a vector of characters and cast each of the characters to their corresponding float value, as per their [ASCII representation](http://www.asciitable.com/)! 

Attempting to cast strings to symbols using the keyed words symbol or 11h will fail with the type error:

In [25]:
`symbol$"4222.001"                 //Invalid syntax 

QError: type

In [ ]:
11h$"4222.001"                     //Invalid syntax

There is a special casting syntax we can use so kdb+/q will interpret the string as a whole to cast -  the capitalization of the character type representation: 

In [26]:
"F"$"4222.001"

4222.001


In the following examples, we used the uppercase letter representing the data type we want returned as the first argument to the `$` operator to convert the second string input:

In [27]:
"J"$"1000"         //Cast a string to a long
"E"$"3283.192"     //Cast a string to a real
"T"$"09:10:15.888" //Cast a string to a time

1000
3283.192e
09:10:15.888


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Careful! In some cases kdb+/q can't interpret the whole string as a feasible representation of the chosen type, e.g. <code>"J"$"120.00"</code> - a whole number doesn't have any decimal points! If it encounters this situation it will return a null of the type we wanted to return - in this particular case <code>0N</code>. </i></p>

##### Exercise  
Cast the string “2020.01.01” to a date 

In [ ]:
"D"$"2020.01.01" //using uppercase to convert the string to date

In [28]:
// Write your code here
"D";"2020.01.01"


2020.01.01


## Symbols
Unlike strings, symbols in kdb+/q are not lists and are actually atomic items. 

### Casting *to* a Symbol

There is actually only *one* datatype that can be cast to a symbol and that is a string (aka a list of characters) .

In [29]:
`$ string .z.d
$[`;string 15.499]

y:("IBM";"MSFT";"GOOG");`$y   //casting from strings

2025.02.17
15.499
`IBM`MSFT`GOOG


##### Exercise

Cast the pi value (3.14159) to a symbol. 

In [ ]:
`$string 3.14159

In [32]:
// Write your code here
type $[`;string 3.14159]

-11h


### Casting *from* a Symbol
In a similar way to casting to a symbol, there is only one type that a Symbol can be cast to - a String, again using the `string` command.

In [33]:
string `IBM`MSFT`GOOG

"IBM"
"MSFT"
"GOOG"


##### Exercise
Cast <code>\`0\`1\`2\`3</code> to a list of longs

In [ ]:
"J"$string `0`1`2`3

In [34]:
// Write your code here
a:`0`1`2
string a

,"0"
,"1"
,"2"
